<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#torch.jit.trace" data-toc-modified-id="torch.jit.trace-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>torch.jit.trace</a></span><ul class="toc-item"><li><span><a href="#调用链路" data-toc-modified-id="调用链路-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>调用链路</a></span></li></ul></li></ul></div>

TorchScript是pytorch为torch.nn.Module设计的一种中间表示形式，JIT解释器可以将torch.nn.Module转换为ScriptModle类型进而保存在磁盘上，从而被不同的环境、非python语言、不同的推理框架加载，提高可应用范围和推理性能。JIT提供了两种方式——trace和script——将torch.nn.Module转换为ScriptModel，这里我们看一下torch.jit.trace的实现代码。

## torch.jit.trace

### 调用链路

torch.jit.trace函数经过层层调用到达了与C++的接口处_create_method_from_trace：
```python
# torch/jit/_trace.py
def trace(
    func,
    example_inputs,
    optimize=None,
    check_trace=True,
    check_inputs=None,
    check_tolerance=1e-5,
    strict=True,
    _force_outplace=False,
    _module_class=None,
    _compilation_unit=_python_cu,
)

-> 

def trace_module(
    mod,
    inputs,
    optimize=None,
    check_trace=True,
    check_inputs=None,
    check_tolerance=1e-5,
    strict=True,
    _force_outplace=False,
    _module_class=None,
    _compilation_unit=_python_cu,
)
  # module is TopLevelTracedModule
  module._c._create_method_from_trace(
                method_name,
                func,
                example_inputs,
                var_lookup_fn,
                strict,
                _force_outplace,
                argument_names,
            )
```

先来看一下pybind绑定函数，可以发现_create_method_from_trace这个函数是绑定在ScriptModule类上，说明上调用它的`module`变量是一个ScriptModule。
```c++
# torch\csrc\jit\python\script_init.cpp
void initJitScriptBindings(PyObject* module):
    py::class_<Module, Object>(m, "ScriptModule")
          .def(
          "_create_method_from_trace",
            ......
            std::shared_ptr<Graph> graph =
                std::get<0>(tracer::createGraphByTracing(
                    func,
                    typed_inputs,
                    var_name_lookup_fn,
                    strict,
                    force_outplace,
                    &self,
                    argument_names));
            ......
          )
```


这个函数再继续往下调用直到遇见了核心函数trace：
```c++
// torch\csrc\jit\frontend\tracer.cpp
std::pair<std::shared_ptr<TracingState>, Stack> trace(
    Stack inputs,
    const std::function<Stack(Stack)>& traced_fn,
    std::function<std::string(const Variable&)> var_name_lookup_fn,
    bool strict,
    bool force_outplace,
    Module* self,
    const std::vector<std::string>& argument_names)
```

这个函数执行了下面几个步骤完成转换：
+ 将torch.nn.Module的输入映射为ScriptModule的输入形式
+ 运行模型得到当前输入下的最终输出
+ 解析并记录每一个算子的输出
+ 对输出的静态图结果进行inline pass优化
+ 对输出的节点进行NormalizeOps优化

具体的解释可以参考：https://zhuanlan.zhihu.com/p/489090393